In [ ]:
import sys
import os
from random import random
from operator import add
from tabulate import tabulate
import logging
import argparse
import pandas as pd
import threading
import csv
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length, regexp_replace, lit
from pyspark.sql.types import *

def pandas_read_csv(file_path,**options):
    """
        Read small volume of data only using read.csv
        Args:
            **Options ----> Any
    """
    try:
        df = pd.read_csv(file_path,**options)
        return df
    except FileNotFoundError:
        print(f"Error: File not found at path: {file_path}")
        return None
    except Exception as e:  # Catch other potential exceptions (e.g., parsing errors)
        print(f"An error occurred while reading the CSV: {e}")
        return None

def loadTable(**kwargs):
    pathCheck = kwargs["path"].replace("/part*","")
    if not os.path.exists(pathCheck):
        return None
    try:
        sparkDqc.sql(f"""
        CREATE EXTERNAL TABLE IF NOT EXISTS {kwargs["tableName"]}
        USING PARQUET LOCATION '{kwargs["path"]}'
        """)
        return True
    except Exception as e:
        return None
    
def ListTable():
    listCreatedTable = []
    tables = sparkDqc.sql("SHOW TABLES").collect()
    for table in tables:
       listCreatedTable.append(table.tableName)
    return listCreatedTable

def getTables(**kwargs):
    df = pandas_read_csv(kwargs["path"], sep="|")
    df = df.query(f"BatchName == '{batchname}' & Run == 1")
    print(tabulate(df.head(), headers='keys', tablefmt='pretty'))
    df_refTable = df.query("RefTable.notnull()")
    distinct_reftable = df_refTable[["BatchName","RefTable"]].drop_duplicates().rename(columns={"RefTable": "JobName"})
    df_job = pandas_read_csv(kwargs["LoadPath"], sep="|")
    df_job = df_job[["BatchName","JobName"]].query(f"BatchName == '{batchname}'")
    joined_df = pd.concat([df_job, distinct_reftable], ignore_index=True).drop_duplicates()
    print(tabulate(joined_df.head(), headers='keys', tablefmt='pretty'))
    return joined_df, df

def executeScripts(sparkDqc, **kwargs):
    try:
        if kwargs["SP"] == 1:
            with open(kwargs["Scripts"], 'r') as file:
                sql_content = file.read()
            sql_statements = sql_content.split(";")
            sql_statements = [s.strip() for s in sql_statements if s.strip()]

            for sql in sql_statements:
                sql_upper = sql.upper()
                if sql_upper.startswith("SELECT"): #Check if the string STARTS with select.
                    result_df = sparkDqc.sql(sql)
                else:
                    sparkDqc.sql(sql)
        else:
            result_df = sparkDqc.sql(kwargs["Scripts"])

        count = result_df.collect()[0]
        kwargs["results"].append((kwargs["BatchName"], kwargs["DqcId"], kwargs["Scripts"], kwargs["Run"], str(count["CNT"])))
    except Exception as e:
        kwargs["results"].append((kwargs["BatchName"], kwargs["DqcId"], kwargs["Scripts"], kwargs["Run"], f"Error : {str(e)}"))

def executeScriptsParallel(sparkDqc, df):
    results = []
    threads = []

    for row in df.itertuples(index=False):
        kwargs = {
            "BatchName": row.BatchName,
            "DqcId": row.DqcId,
            "Scripts": row.Scripts,
            "Run": row.Run,
            "SP": row.SP,
            "results": results
        }
         
        thread = threading.Thread(target=executeScripts, args=(sparkDqc,), kwargs=kwargs)
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    result_df = sparkDqc.createDataFrame(results, ["BatchName", "DqcId", "Scripts", "Run", "Result"])
    return result_df

if __name__ == "__main__":
    path = "/mnt/apps/Files/Config/DataQuality_Config.csv"
    LoadPath = "/mnt/apps/Files/Config/master_job.csv"
    parquetOutput = "/mnt/apps/Files/data-movement/Parquet/"
    batchname = "BATCH_ACT_VAL"
    testWrite = "/mnt/apps/Files/Test"

    getDf, df = getTables(path=path, LoadPath=LoadPath)

    sparkDqc = (
        SparkSession
            .builder
            .appName(f"{batchname}_DQC")
            .master("local[*]")
            .config("spark.ui.port", "4222")
            .getOrCreate()
    )

    listJob = []
    for row in getDf.itertuples():
        listJob.append(row.JobName.lower())
        loadTable(path=parquetOutput + '/' + row.JobName + '/part*', tableName=row.JobName)
        
    print(ListTable())
    print(listJob)
    
    if all(item in ListTable() for item in listJob):
        result_df = executeScriptsParallel(sparkDqc, df)
        print(result_df.show(truncate=False))
    else:
        sys.exit(1)

    sparkDqc.stop()

In [ ]:
from pyspark.sql import SparkSession

spark = (
        SparkSession
        .builder
        .appName("Locally")
        .master("local[*]")
        .config("spark.ui.port", "4222")
        .config("spark.executor.instances", "2")  # Initial number of executors (if dynamicAllocation is false)
        .config("spark.executor.cores", "2")  # Cores per executor
        .config("spark.cores.max", "3") # Maximum total cores for the application
        .config("spark.dynamicAllocation.enabled", "true") #Enable dynamic allocation
        .config("spark.dynamicAllocation.minExecutors", "1") #minimum executors
        .config("spark.dynamicAllocation.maxExecutors", "5") #maximum executors
        .getOrCreate()
)

spark

In [9]:
spark.stop()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import explode

def dict_to_row(data_dict):
    """Converts a dictionary to a Spark Row."""
    return Row(**data_dict)

def create_dataframe_from_dict(spark, data_dict):
    """Creates a Spark DataFrame from a dictionary."""
    row = dict_to_row(data_dict)
    return spark.createDataFrame([row])

if __name__ == "__main__":
    spark = SparkSession.builder.appName("DictToDataFrame").getOrCreate()

    data = {
        'JobName': 'RTRNPF',
        'Path': '/mnt/apps/gcs/ETL4/RTRNPF/RTRNPF*.csv',
        'dqID': 'DQ000001',
        'CountRecords': -1,
        'Message': ['Invalid Index values (containing only non-numeric characters). Total count: 5', "Invalid Subscription Date values (not in '%Y-%m-%d' format). Total count: 1"],
        'Status': 'Failed'
    }

    df = create_dataframe_from_dict(spark, data)

    # Explode the 'Message' array into rows
    exploded_df = df.select("JobName", "Path", "dqID", "CountRecords", explode("Message").alias("Message"), "Status")

    exploded_df.show(truncate=False)
    exploded_df.printSchema()

    spark.stop()

In [ ]:
test_Split = ("CREATE TEMPORARY VIEW temp_jobs AS SELECT `Job Title`,COUNT(1) CNT FROM PEOPLEPF WHERE YEAR(`Date of birth`) IN ('1992','1993','1995','2021') GROUP BY `Job Title`; SELECT COUNT(1) CNT FROM temp_jobs").split(";")
print(test_Split)

for scripts in test_Split:
    scripts = scripts.strip()
    print(scripts)

In [ ]:
from pyspark.sql import SparkSession
import csv

spark = SparkSession.builder.appName("CSVMultiLine").getOrCreate()

# Sample DataFrame (replace with your actual data source)
data = [("John", "Engineer", "1992-05-15"), ("Jane", "Manager", "1993-10-20"), ("Mike", "Engineer", "1995-03-01"), ("Sarah", "Analyst", "2021-12-05")]
columns = ["Name", "Job Title", "Date of birth"]
people_df = spark.createDataFrame(data, columns)
people_df.createOrReplaceTempView("PEOPLEPF")

def execute_sql_from_csv(csv_file_path):
    with open(csv_file_path, 'r') as file:
        sql_content = file.read()
    sql_statements = sql_content.split(";")
    sql_statements = [s.strip() for s in sql_statements if s.strip()]
    for sql in sql_statements:
        sql_upper = sql.upper()
        if sql_upper.startswith("SELECT"):
            result_df = spark.sql(sql)
            result_df.show()
        else:
            spark.sql(sql)

csv_file_path = '/mnt/apps/Files/SP/USP_FCT.sql'

execute_sql_from_csv(csv_file_path=csv_file_path)
            